# AI Video GPU Platform - Kaggle Setup

This notebook sets up the AI Video GPU platform on Kaggle with free GPU access.

## Prerequisites
- Kaggle account (free)
- Phone verification for GPU access
- Enable "Internet" and "GPU" in notebook settings

## GPU Availability
- **Free Tier**: Up to 30 hours/week of GPU time
- **GPU Types**: T4 x2, P100 (depending on availability)
- **RAM**: 13-16GB
- **Storage**: 20GB temporary

⚠️ **Important**: Make sure to enable GPU in notebook settings (Accelerator > GPU T4 x2)

## Step 1: System Information & GPU Check

In [ ]:
import os
import sys
import subprocess
import torch

print("🔍 System Information:")
print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Memory: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ GPU not detected! Please enable GPU in notebook settings.")
    print("Go to Settings > Accelerator > GPU T4 x2")

# Check available disk space
result = subprocess.run(['df', '-h', '/kaggle/working'], capture_output=True, text=True)
print(f"\n💾 Disk space:\n{result.stdout}")

## Step 2: Install System Dependencies

In [ ]:
# Update system packages
!apt-get update -qq

# Install essential system packages
!apt-get install -y -qq \
    ffmpeg \
    libsm6 \
    libxext6 \
    libxrender-dev \
    libglib2.0-0 \
    libgl1-mesa-glx \
    git \
    wget \
    curl

print("✅ System dependencies installed successfully!")

## Step 3: Clone Repository

In [ ]:
import os

# Change to working directory
os.chdir('/kaggle/working')

# Clone the repository
if not os.path.exists('AI-Video-GPU'):
    !git clone https://github.com/yourusername/AI-Video-GPU.git
    print("✅ Repository cloned successfully!")
else:
    print("📁 Repository already exists")

# Change to project directory
os.chdir('/kaggle/working/AI-Video-GPU')
print(f"📍 Current directory: {os.getcwd()}")

# List contents to verify
!ls -la

## Step 4: Install Python Dependencies

In [ ]:
# Upgrade pip
!pip install --upgrade pip

# Install main requirements
!pip install -r requirements.txt

# Install additional packages for Kaggle environment
!pip install gradio pyngrok

print("✅ Python dependencies installed successfully!")

## Step 5: Configure Environment

In [ ]:
import os

# Set environment variables for Kaggle
os.environ['KAGGLE_ENV'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'  # Use both GPUs if available
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

# Set paths
os.environ['PROJECT_ROOT'] = '/kaggle/working/AI-Video-GPU'
os.environ['MODEL_CACHE'] = '/kaggle/working/models'
os.environ['OUTPUT_DIR'] = '/kaggle/working/outputs'

# Create necessary directories
os.makedirs('/kaggle/working/models', exist_ok=True)
os.makedirs('/kaggle/working/outputs', exist_ok=True)
os.makedirs('/kaggle/working/temp', exist_ok=True)

print("✅ Environment configured for Kaggle!")
print(f"📁 Models cache: {os.environ['MODEL_CACHE']}")
print(f"📁 Output directory: {os.environ['OUTPUT_DIR']}")

## Step 6: Download Models (Optional - Heavy Downloads)

In [ ]:
# This cell downloads AI models. Uncomment and run if needed.
# Note: Models are large and may take significant time/bandwidth

# import torch
# from huggingface_hub import snapshot_download
# 
# print("🔄 Downloading models... This may take several minutes")
# 
# # Download Stable Diffusion model
# try:
#     model_path = snapshot_download(
#         repo_id="runwayml/stable-diffusion-v1-5",
#         cache_dir="/kaggle/working/models",
#         allow_patterns=["*.bin", "*.json", "*.txt"]
#     )
#     print(f"✅ Stable Diffusion downloaded to {model_path}")
# except Exception as e:
#     print(f"⚠️ Error downloading Stable Diffusion: {e}")
# 
# # Download other models as needed
# # Add more model downloads here

print("💡 Model download cell ready. Uncomment code above to download models.")
print("⚠️ Note: Models are large and will consume significant time and bandwidth.")

## Step 7: Set up Ngrok for Public Access (Optional)

In [ ]:
# Optional: Set up ngrok for public access
# Get your free ngrok token from https://ngrok.com/

NGROK_TOKEN = ""  # Enter your ngrok token here

if NGROK_TOKEN:
    from pyngrok import ngrok
    
    # Set ngrok auth token
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ Ngrok configured for public access")
else:
    print("💡 Ngrok token not provided. App will run locally only.")
    print("Get your free token at: https://ngrok.com/")

## Step 8: Launch the Application

In [ ]:
import sys
import os

# Add project to Python path
sys.path.append('/kaggle/working/AI-Video-GPU')

try:
    # Import and launch the main application
    from main import create_gradio_interface
    
    # Create the Gradio interface
    demo = create_gradio_interface()
    
    # Launch with Kaggle-specific settings
    if 'NGROK_TOKEN' in globals() and NGROK_TOKEN:
        # Launch with ngrok for public access
        demo.launch(
            share=True,
            server_name="0.0.0.0",
            server_port=7860,
            debug=True
        )
    else:
        # Launch locally (Kaggle internal only)
        demo.launch(
            share=False,
            server_name="0.0.0.0",
            server_port=7860,
            debug=True,
            inline=True  # Show inline in Kaggle
        )

except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Running basic demo instead...")
    
    # Fallback: Basic Gradio demo
    import gradio as gr
    
    def basic_demo(text):
        return f"AI Video GPU Platform is running! Input: {text}"
    
    demo = gr.Interface(
        fn=basic_demo,
        inputs=gr.Textbox(label="Test Input"),
        outputs=gr.Textbox(label="Output"),
        title="AI Video GPU - Kaggle Demo",
        description="Basic demo running on Kaggle GPU environment"
    )
    
    demo.launch(
        share=False,
        server_name="0.0.0.0",
        server_port=7860,
        inline=True
    )

## Step 9: Test GPU Performance

In [ ]:
import torch
import time

if torch.cuda.is_available():
    print("🚀 Testing GPU performance...")
    
    # Test matrix multiplication on GPU
    device = torch.device('cuda')
    
    # Create large tensors
    size = 4096
    a = torch.randn(size, size, device=device)
    b = torch.randn(size, size, device=device)
    
    # Warm up
    _ = torch.matmul(a, b)
    torch.cuda.synchronize()
    
    # Benchmark
    start_time = time.time()
    result = torch.matmul(a, b)
    torch.cuda.synchronize()
    end_time = time.time()
    
    print(f"✅ GPU matrix multiplication ({size}x{size}): {end_time - start_time:.3f} seconds")
    print(f"🔥 GPU memory used: {torch.cuda.memory_allocated()/1024**3:.2f} GB")
    print(f"🔥 GPU memory cached: {torch.cuda.memory_reserved()/1024**3:.2f} GB")
    
    # Clear memory
    del a, b, result
    torch.cuda.empty_cache()
    
else:
    print("❌ GPU not available for testing")

## Usage Tips for Kaggle

### 📝 Important Notes:
1. **GPU Quota**: You have 30 hours/week of free GPU time
2. **Session Limits**: Sessions auto-terminate after 9 hours of inactivity
3. **Storage**: 20GB temporary storage, data is lost when session ends
4. **Internet**: Must be enabled in notebook settings for downloads

### 💡 Best Practices:
- Save important outputs to Kaggle Datasets for persistence
- Monitor GPU usage in the right panel
- Use smaller models for quick testing
- Enable version control to save notebook states

### 🔧 Troubleshooting:
- **No GPU**: Enable GPU in Settings → Accelerator
- **Out of Memory**: Reduce batch sizes or use gradient checkpointing
- **Slow Downloads**: Large models may timeout, try smaller variants
- **Session Timeout**: Re-run cells to restore state

### 📊 Monitoring Resources:
```python
# Check GPU memory
!nvidia-smi

# Check disk usage
!df -h

# Check RAM usage
!free -h
```

## 🎉 Setup Complete!

Your AI Video GPU platform is now running on Kaggle! 

### Next Steps:
1. Access the Gradio interface above
2. Upload your media files
3. Configure generation settings
4. Generate AI videos and content

### Save Your Work:
- Use "Save Version" to preserve your notebook
- Export outputs to Kaggle Datasets
- Download important files before session ends

Happy creating! 🚀